In [ ]:
import pandas as pd

# Read data
#data = pd.read_excel('data.xlsx', sheet_name='PS19A_PERSON__SmplR10K', keep_default_na=False)
data = pd.read_csv('data_small.tsv.win.utf8.txt', sep='\t', keep_default_na=False)

# Extract addresses
addresses = data['person_address']
del data

addressCount = len(addresses)

data = {}


In [ ]:
import Levenshtein as lev
import multiprocessing
import threading
import math

# Number of threads to create
threadCount = multiprocessing.cpu_count()

# Amount of addresses to use per thread
threadAddressCount = int(math.ceil(float(addressCount) / threadCount))

class CalculateDistanceThread(threading.Thread):
    def __init__(self, threadID):
        threading.Thread.__init__(self)
        self.threadID = threadID
    def run(self):
        start = threadAddressCount * self.threadID
        for n in range(threadAddressCount):
            if (start + n < addressCount):
                for x in range(addressCount):
                    ratio = lev.ratio(addresses[n].lower(), addresses[x].lower())
                    if (ratio > 0.6):
                        data[(n, x)] = ratio

# Create a list with n threads
threads = []
for n in range(threadCount):
    thread = CalculateDistanceThread(n)
    threads.append(thread)

# Start all threads
for thread in threads:
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

In [ ]:
from pprint import pprint

pprint(list(data.items())[:8])